In [15]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### 5.3.1 The Validation Set Approach

In [16]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [17]:
def evalMSE(terms, response, train, test):
  mm = MS(terms)
  X_train = mm.fit_transform(train)
  y_train = train[response]
  X_test = mm.transform(test)
  y_test = test[response]
  results = sm.OLS(y_train, X_train).fit()
  test_pred = results.predict(X_test)
  return np.mean((y_test - test_pred)**2)

MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
  MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [18]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
  MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

### 5.3.2 Cross-Validation

In [19]:
# Custom wrapper function provided by ISLP library for transfering statsmodels model to sklearn model 
hp_model = sklearn_sm(sm.OLS, MS(['horsepower'])) 
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929226

In [20]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
  X = np.power.outer(H, np.arange(d+1))
  M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
  cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443032, 19.03323636])

In [21]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10, shuffle=True, random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
  X = np.power.outer(H, np.arange(d+1))
  M_CV = cross_validate(M, X, Y, cv=cv)
  cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13719339])

In [22]:
validation = ShuffleSplit(n_splits=1, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation)
results['test_score']

array([23.61661707])

In [23]:
validation = ShuffleSplit(n_splits=10, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.421845094109187)

### 5.3.3 The Bootstrap

Estimating the Accuracy of a Statistic of Interest

In [24]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
  cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
  return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

alpha_func(Portfolio, range(100))

0.5758320745928298

In [25]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

0.6074452469619002

In [40]:
def boot_SE(func, D, n=None, B=1000, seed=0):
  rng = np.random.default_rng(seed)
  first_, second_ = 0, 0
  n = n or D.shape[0]
  for _ in range(B):
    idx = rng.choice(D.index, n, replace=True)
    value = func(D, idx)
    first_ += value
    second_ += value**2
  return np.sqrt(second_ / B - (first_ / B)**2)

alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277577

Estimating the Accuracy of a Linear Regression Model

In [43]:
def boot_OLS(model_matrix, response, D, idx):
  D_ = D.iloc[idx]
  Y_ = D_[response]
  X_ = clone(model_matrix).fit_transform(D_)
  return sm.OLS(Y_, X_).fit().params

hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')  # !!!

Auto_reset = Auto.reset_index()

rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(392, 392, replace=True)) for _ in range(10)])

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [44]:
hp_se = boot_SE(hp_func, Auto_reset, B=1000, seed=10)
hp_se

intercept     0.848807
horsepower    0.007352
dtype: float64

In [45]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [47]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto_reset, B=1000)

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [48]:
M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64